# Scanning Session Ingestion

## Setup

### Connect to the database

If you are don't have your login information, contact the administrator.

Using local config file (see [01_pipeline](./01_pipeline.ipynb)):

In [10]:
import os
# change to the upper level folder to detect dj_local_conf.json
if os.path.basename(os.getcwd())=='notebooks': os.chdir('..')
assert os.path.basename(os.getcwd())=='adamacs', ("Please move to the main directory")
import datajoint as dj; dj.conn()

DataJoint connection (connected) tobiasr@172.26.128.53:3306

Manual entry:

In [ ]:
# Manual Entry
import datajoint as dj; import getpass
dj.config['database.host'] = '172.26.128.53'        # Put the server name between these apostrophe
dj.config['database.user'] = 'danielmk'             # Put your user name between these apostrophe
dj.config['database.password'] = getpass.getpass()  # Put your password in the prompt
dj.conn()

### Imports and activation

Importing schema from `adamacs.pipeline` automatically activates relevant schema.

In [11]:
import datajoint as dj
from adamacs.pipeline import subject, session, surgery, scan, event, trial
from adamacs import utility
from adamacs.ingest import behavior as ibe

Assign easy names for relevant tables

In [12]:
sub, lab, protocol, line, mutation, user, project, subject_genotype, subject_death = (
    subject.Subject(), subject.Lab(), subject.Protocol(), subject.Line(), 
    subject.Mutation(), subject.User(), subject.Project(), subject.SubjectGenotype(), 
    subject.SubjectDeath())

In [13]:
event_types = ['main_track_gate', 'shutter', 'mini2p_frames', 'mini2p_lines', 'mini2p_volumes', 'aux_bpod_cam',
               'aux_bpod_visual', 'aux_bpod_reward', 'aux_bpod_tone']

for e in event_types:
    event.EventType.insert1({'event_type': e, 'event_type_description': ''}, skip_duplicates=True,)

In [15]:
session.Session()

*session_id    subject      session_dateti
+------------+ +----------+ +------------+
sess9FB2LN5C   WEZ-8701     2022-03-18 00:
sess9FF6U114   ROS-1438     2022-11-22 00:
 (Total: 2)

## Ingesting behavior

In [24]:
ibe.ingest_aux("sess9FF6U114",verbose=True)

Number of scans found: 1


In [18]:
event.Event()

*session_id    *event_type    *event_start_t event_end_time
+------------+ +------------+ +------------+ +------------+
sess9FF6U114   aux_bpod_cam   0.0            0.005         
sess9FF6U114   aux_bpod_cam   0.01665        0.02165       
sess9FF6U114   aux_bpod_cam   0.03335        0.03835       
sess9FF6U114   aux_bpod_cam   0.05           0.055         
sess9FF6U114   aux_bpod_cam   0.06665        0.07165       
sess9FF6U114   aux_bpod_cam   0.08335        0.08835       
sess9FF6U114   aux_bpod_cam   0.1            0.105         
sess9FF6U114   aux_bpod_cam   0.11665        0.12165       
sess9FF6U114   aux_bpod_cam   0.13335        0.13835       
sess9FF6U114   aux_bpod_cam   0.15           0.155         
sess9FF6U114   aux_bpod_cam   0.16665        0.17165       
sess9FF6U114   aux_bpod_cam   0.18335        0.18835       
   ...
 (Total: 649715)

In [ ]:
isess.ingest_session_scan('sess9FB2LN5C', verbose=True)

In [ ]:
session.Session * session.SessionDirectory

In [31]:
key='scan9FF6U114'
(scan.Scan & f'scan_id=\"{key}\"')

*session_id    *scan_id       scanner     acq_software   scan_notes    
+------------+ +------------+ +---------+ +------------+ +------------+
sess9FF6U114   scan9FF6U114   mini2p1     ScanImage                    
 (Total: 1)

In [30]:
scan.Scan()

*session_id    *scan_id       scanner     acq_software   scan_notes    
+------------+ +------------+ +---------+ +------------+ +------------+
sess9FF6U114   scan9FF6U114   mini2p1     ScanImage                    
 (Total: 1)

In [26]:
scan.ScanInfo.populate()

`CB DEV NOTE:` In demo file, there are no `scan.motor_position_at_zero` values. Set to 0 here to avoid downstream type errors when adding. Future would should set these to null when not present and revise downstream code.

In [28]:
scan.ScanInfo()

*session_id    *scan_id       nfields     nchannels     ndepths     nframes     nrois     x       y       z       fps         bidirectional  usecs_per_line fill_fraction  scan_datetime  scan_duration 
+------------+ +------------+ +---------+ +-----------+ +---------+ +---------+ +-------+ +-----+ +-----+ +-----+ +---------+ +------------+ +------------+ +------------+ +------------+ +------------+
sess9FF6U114   scan9FF6U114   1           1             1           9250        0         nan     nan     nan     15.3845     1              250.003        0.95           None           601.255       
 (Total: 1)

In [32]:
scan.ScanInfo.Field()

*session_id    *scan_id       *field_idx    px_height     px_width     um_height     um_width     field_x     field_y     field_z     delay_imag roi     
+------------+ +------------+ +-----------+ +-----------+ +----------+ +-----------+ +----------+ +---------+ +---------+ +---------+ +--------+ +------+
sess9FF6U114   scan9FF6U114   0             256           256          65.7966       65.7966      nan         nan         nan         =BLOB=     None    
 (Total: 1)

Note the relative path below:

In [33]:
scan.ScanInfo.ScanFile()

*session_id    *scan_id       *file_path    
+------------+ +------------+ +------------+
sess9FF6U114   scan9FF6U114   /Users/trose/D
 (Total: 1)